### Intial Testing of the Dataset

In [ ]:
import numpy as np
import pandas as pd 
import torch
import torch.nn as nn
import os 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sentence_transformers import SentenceTransformer

ModuleNotFoundError: No module named 'sentence_transformer'

In [53]:
df = pd.read_csv('data/imdb_top_1000.csv')
df.at[966,"Released_Year"] = 1995 #fixing the wrong value for apollo 13 
df.drop(columns = ["Poster_Link"], inplace= True)
df.dropna(inplace = True) #dropping null valued columns


In [54]:
#Converting to Numeric Values
df["Gross"] = df["Gross"].str.replace(",", "").astype("float")
df["Released_Year"] = df["Released_Year"].astype("int")
df["Runtime"] = df["Runtime"].str.replace(" min", "").astype("int")
df["IMDB_Rating"] = df["IMDB_Rating"].astype(float)
df["Meta_score"] = df["Meta_score"].astype(float)
df["No_of_Votes"] = df["No_of_Votes"].astype(float)
df.head()


,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,The Shawshank Redemption,1994,A,142,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110.0,28341469.0
1,The Godfather,1972,A,175,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367.0,134966411.0
2,The Dark Knight,2008,UA,152,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232.0,534858444.0
3,The Godfather: Part II,1974,A,202,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952.0,57300000.0
4,12 Angry Men,1957,U,96,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845.0,4360000.0


In [ ]:
df["IMDB_Rating"] = smooth_target_encode(df, "IMDB_Rating", "Gross")
df["Meta_score"] = smooth_target_encode(df, "Meta_score", "Gross")
df["No_of_Votes"] = smooth_target_encode(df, "No_of_Votes", "Gross")
df["Certificate"] = smooth_target_encode(df, "Certificate", "Gross")
df["Genre"] = smooth_target_encode(df, "Genre", "Gross")
df["Director"] = smooth_target_encode(df, "Director", "Gross")
df["Star1"] = smooth_target_encode(df, "Star1", "Gross")
df["Star2"] = smooth_target_encode(df, "Star2", "Gross")
df["Star3"] = smooth_target_encode(df, "Star3", "Gross")
df["Star4"] = smooth_target_encode(df, "Star4", "Gross")
df["Series_Title"] = smooth_target_encode(df, "Series_Title", "Gross")




### Trying One Hot Encoding
Thoughts - Too many features \
Results - Not Tried

In [58]:
#One Hot encoding for Certificate, Genre, Director, Star1, Star2, Star3, Star 4
df_encoded = pd.get_dummies(df, columns = ["Certificate", "Genre", "Director", "Star1", "Star2", "Star3", "Star4"])
print(f"df_encoded_shape{df_encoded.shape}")
print(f"df_shape{df.shape}")

df_encoded_shape(714, 2962)
df_shape(714, 15)


### Trying Target Encoding

In [ ]:
#defining functino to encode the target variable
def smooth_target_encode(df, col, target, smoothing_param =.3):
    mean_target = df[target].mean()
    encoded =df.groupby(col)[target].agg(["count", "mean"])
    counts = encoded["count"]
    means = encoded["mean"]
    smooth_encodings = (means*counts + mean_target*smoothing_param)/(counts+smoothing_param)
    return df[col].map(smooth_encodings)

In [69]:
df["IMDB_Rating"] = smooth_target_encode(df, "IMDB_Rating", "Gross")
df["Meta_score"] = smooth_target_encode(df, "Meta_score", "Gross")
df["No_of_Votes"] = smooth_target_encode(df, "No_of_Votes", "Gross")
df["Certificate"] = smooth_target_encode(df, "Certificate", "Gross")
df["Genre"] = smooth_target_encode(df, "Genre", "Gross")
df["Director"] = smooth_target_encode(df, "Director", "Gross")
df["Star1"] = smooth_target_encode(df, "Star1", "Gross")
df["Star2"] = smooth_target_encode(df, "Star2", "Gross")
df["Star3"] = smooth_target_encode(df, "Star3", "Gross")
df["Star4"] = smooth_target_encode(df, "Star4", "Gross")
df["Series_Title"] = smooth_target_encode(df, "Series_Title", "Gross")




In [70]:
df.head()

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,3.991965e+07,1994,6.605757e+07,142,3.478698e+07,3.991965e+07,Two imprisoned men bond over a number of years...,1.049399e+08,8.204214e+07,3.991965e+07,3.991965e+07,3.991965e+07,3.991965e+07,3.991965e+07,28341469.0
1,1.219388e+08,1972,6.605757e+07,175,3.328097e+07,1.219388e+08,An organized crime dynasty's aging patriarch t...,2.578054e+07,6.988265e+07,7.309586e+07,5.893506e+07,1.219388e+08,9.383499e+07,1.219388e+08,134966411.0
2,4.295481e+08,2008,1.499447e+08,152,7.275734e+07,1.879008e+08,When the menace known as the Joker wreaks havo...,8.741143e+07,2.362660e+08,1.656802e+08,2.788940e+08,4.295481e+08,1.803062e+08,4.295481e+08,534858444.0
3,6.219544e+07,1974,6.605757e+07,202,3.328097e+07,1.879008e+08,The early life and career of Vito Corleone in ...,7.654053e+07,6.988265e+07,4.794513e+07,1.465885e+08,7.144591e+07,9.383499e+07,6.219544e+07,57300000.0
4,2.147237e+07,1957,9.459847e+07,96,3.328097e+07,1.879008e+08,A jury holdout attempts to prevent a miscarria...,6.266548e+07,3.760792e+07,1.008806e+07,1.110124e+07,1.403221e+07,1.110124e+07,1.110124e+07,4360000.0
